In [1]:
!uv add pydantic openai dotenv jsonschema

Resolved 58 packages in 0.79ms
Audited 23 packages in 0.16ms


In [1]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Callable, Any
from openai import OpenAI
from os import getenv
from dotenv import load_dotenv

import inspect
import httpx

import os
import json
import jsonschema
from jsonschema import Draft7Validator

from copy import deepcopy
from pprint import pprint as pp

In [2]:
def basic_type_converter(t):
    if t == 'str':
        return 'string'
    if t == 'int' or t == 'float':
        return 'number'
    if t == 'list':
        return 'array'
    #TODO: more complicated processing for dictionary types

In [3]:
def get_func_desc(func):
    desc = ''
    vals = []
    for x in deepcopy(func.__doc__).split('\n'):
        if x != '':
            vals.append(x.strip())
    idx = vals.index('Args:')
    for x in vals[:idx]:
        desc += x
    return desc

In [4]:
class ToolDefinition(BaseModel):
    function: Callable
    name: str = Field(default_factory=lambda data: data['function'].__name__)
    desc: str = Field(default_factory=lambda data: get_func_desc(data['function']))

    #TODO: fix no-argument, no-default tools not being supported
    def format_props(self):
        raw_props = []
        for x in deepcopy(self.function.__doc__.split('\n')):
            if x != '':
                raw_props.append(x.strip())
        idx = raw_props.index('Args:')+1
        props = {}
        for arg in raw_props[idx:]:
            name, ty, des = [x.strip() for x in arg.split(':')]
            props[name] = {"type": basic_type_converter(ty), "description": des}
        return props

    def format_req(self):
        ret = deepcopy(list(self.function.__annotations__.keys()))
        if 'return' in ret:
            ret.remove('return')
        return ret
    
    def format_tool(self):
        return json.dumps({
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.desc,
                "strict": True,
                "parameters": {
                    "type": "object",
                    "properties": self.format_props(),
                    "required": self.format_req(),
                    "additionalProperties": False
                }
            }
        })

In [5]:
def read_file(path: str) -> str:
    """
    Reads the content of a file.
    Args: 
    path: str: the path of the file to be read
    """
    try:
        with open(path, 'r') as f:
            return f.read()
    except Exception as e:
        return f"Error: {str(e)}"

In [6]:
def list_files(path: str = "") -> list:
    """
    Lists the files in a directory.
    Defaults to the current working directory.
    Args:
    path: str: the directory path to list files from. Defaults to cwd if empty.
    """
    target = path or "."
    try:
        entries = os.listdir(target)
        return [
            f"{entry}" if os.path.isdir(os.path.join(target, entry)) else entry
            for entry in entries
        ]
    except Exception as e:
        return [f"Error: {str(e)}"]

In [7]:
def edit_file(path: str, old: str, new: str) -> str:
    """
    Replaces occurrences of old with new in file at path.
    Creates file if it does not exist.
    Args:
    path: str: the file path to edit or create.
    old: str: the substring to be replaced.
    new: str: the new substring to replace the old.
    """
    try:
        # Create file if it doesn't exist
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            with open(path, 'w') as f:
                f.write("")  # Create empty file
            return f"Created {path}"
        
        # Read existing file
        with open(path, 'r') as f:
            content = f.read()
        
        # Replace old with new
        new_content = content.replace(old, new)
        
        # Check if replacement actually happened
        if content == new_content:
            return "Error: old not found"
        
        # Write back to file
        with open(path, 'w') as f:
            f.write(new_content)
        
        return "Edit successful"
        
    except Exception as e:
        return f"Error: {str(e)}"

In [8]:
class Agent(BaseModel):
    model_config=ConfigDict(arbitrary_types_allowed=True)

    client: OpenAI
    model: str #TODO: validate against openrouter API
    tools: dict[str, ToolDefinition]
    conversation: list[dict] = []

    def build_tool_schema(self):
        schemas = []
        for tool in self.tools.values():
            schemas.append(json.loads(tool.format_tool()))
        return schemas

    def user_message(self, user_in):
        return {
                "role": "user",
                "content": [{"type": "text", "text": user_in}]
            }
    
    def run(self):
        print(f"Chat with {self.model} (use 'quit' to quit)")
        while True:
            user_input = input("\033[94mYou: \033[0m").strip()
            if user_input.lower() == "quit":
                break

            self.conversation.append(self.user_message(user_input))

            response = self.get_llm_response()
            self.handle_response(response)

    def get_llm_response(self) -> 'openai.types.chat.chat_completion_message.ChatCompletionMessage':
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.conversation,
            tools=self.build_tool_schema(),
            tool_choice="auto"
        )
        self.conversation.append(response.choices[0].message.model_dump())
        return response

    def execute_single_tool_call(self, tool_call):
        """Execute a tool call and return the formatted response"""
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        #check if tool exists
        if tool_name not in self.tools:
            return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_name,
            "content": f"Error: Unknown tool:{tool_name}",
        }
        #execute
        tool_func = self.tools[tool_name].function
        try:
            tool_result = tool_func(**tool_args)
        except Exception as e:
            tool_result = f"Tool execution error: {str(e)}"
            
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_name,
            "content": str(tool_result),
        }

    def handle_response(self, response):
        if response.choices[0].message.tool_calls:
            for tool_call in response.choices[0].message.tool_calls:
                tool_response = self.execute_single_tool_call(tool_call)
                self.conversation.append(tool_response)
            
        
            # Get another response from the LLM with the tool results
            new_response = self.get_llm_response()
            # Recursively handle the new response (in case it has more tool calls)
            self.handle_response(new_response)
        else:
            # No tool calls - this is the final response
            final_content = response.choices[0].message.content
            print(f"\033[92mAgent: \033[0m{final_content}")

In [24]:
# little bit of how you get the list of models
models_url = "https://openrouter.ai/api/v1/models"
model_resp = httpx.get(models_url)
raw_json = json.loads(model_resp.read())
ids_and_slugs = [{'id': x['id'], 'canonical_slug': x['canonical_slug']} for x in raw_json['data']]
ids_and_slugs

[{'id': 'switchpoint/router', 'canonical_slug': 'switchpoint/router'},
 {'id': 'moonshotai/kimi-k2:free', 'canonical_slug': 'moonshotai/kimi-k2'},
 {'id': 'moonshotai/kimi-k2', 'canonical_slug': 'moonshotai/kimi-k2'},
 {'id': 'thudm/glm-4.1v-9b-thinking',
  'canonical_slug': 'thudm/glm-4.1v-9b-thinking'},
 {'id': 'mistralai/devstral-medium',
  'canonical_slug': 'mistralai/devstral-medium-2507'},
 {'id': 'mistralai/devstral-small',
  'canonical_slug': 'mistralai/devstral-small-2507'},
 {'id': 'cognitivecomputations/dolphin-mistral-24b-venice-edition:free',
  'canonical_slug': 'venice/uncensored'},
 {'id': 'x-ai/grok-4', 'canonical_slug': 'x-ai/grok-4-07-09'},
 {'id': 'google/gemma-3n-e2b-it:free',
  'canonical_slug': 'google/gemma-3n-e2b-it'},
 {'id': 'tencent/hunyuan-a13b-instruct:free',
  'canonical_slug': 'tencent/hunyuan-a13b-instruct'},
 {'id': 'tencent/hunyuan-a13b-instruct',
  'canonical_slug': 'tencent/hunyuan-a13b-instruct'},
 {'id': 'tngtech/deepseek-r1t2-chimera:free',
  'can

In [25]:
load_dotenv()

# gets API Key from environment variable OPENROUTER_API_KEY
cli = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_API_KEY"),
)
test_model = 'anthropic/claude-sonnet-4'
cat = ToolDefinition(function=read_file)
ls = ToolDefinition(function=list_files)
ed = ToolDefinition(function=edit_file)
tool_dic = {"read_file": cat, "list_files": ls, "edit_file": ed}
AUgent = Agent(client=cli, model=test_model, tools=tool_dic)

In [12]:
raw_models = model_resp.json()['data']
model_info = [{x['id']: x['pricing']} for x in raw_models]
#gets all anthropic models
#pp([x for x in model_info if 'anthropic' in list(x.keys())[0]])

In [26]:
AUgent.run()

Chat with anthropic/claude-sonnet-4 (use 'quit' to quit)
Agent: Based on my analysis of the Python file, this is an AI agent framework that provides basic file system operations. Currently, it has three tools:

1. `read_file` - reads file content
2. `list_files` - lists directory contents  
3. `edit_file` - replaces text in files

Here are some additional tools I'd suggest adding to enhance the agent's capabilities:

## File System Tools
```python
def create_file(path: str, content: str) -> str:
    """
    Creates a new file with specified content.
    Args:
    path: str: the file path to create
    content: str: the content to write to the file
    """

def delete_file(path: str) -> str:
    """
    Deletes a file or directory.
    Args:
    path: str: the file or directory path to delete
    """

def copy_file(source: str, destination: str) -> str:
    """
    Copies a file from source to destination.
    Args:
    source: str: the source file path
    destination: str: the destina